## Mouse Control Using Eye LandMarks

### step 1: Import Libraries

In [2]:
import cv2
import mediapipe as mp
import pyautogui

### step 2: Initialize Webcam

In [3]:
# Initialize webcam and face mesh
cam = cv2.VideoCapture(0)
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

### step 3: Control Mouse

In [4]:
# Get screen size
screen_w, screen_h = pyautogui.size()

while True:
    # Read frame from webcam
    _, frame = cam.read()
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = face_mesh.process(rgb_frame)
    landmarks_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    if landmarks_points:
        landmarks = landmarks_points[0].landmark

        for id, landmark in enumerate(landmarks[474:478]):
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)
            
            if id == 1:
                # Convert eye position to screen coordinates
                screen_x = screen_w / frame_w * x
                screen_y = screen_h / frame_h * y
                
                # Ensure coordinates are within screen bounds
                screen_x = min(max(screen_x, 0), screen_w - 1)
                screen_y = min(max(screen_y, 0), screen_h - 1)
                
                pyautogui.moveTo(screen_x, screen_y)

        # Track two specific landmarks to detect eye blinking
        main = [landmarks[145], landmarks[159]]
        
        for landmark in main:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)
        
        # Detect blink based on distance between two landmarks
        if abs(main[0].y - main[1].y) < 0.004:
            pyautogui.click()
            pyautogui.sleep(1)

    # Display the frame with visual markers
    cv2.imshow("Camera", frame)
    
    # Break the loop and stop the camera when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cam.release()
cv2.destroyAllWindows()
